#### This assignment may be worked individually or in pairs. Enter your name/s here:
    

In [62]:
# names here
# Alan Tran 
# John R. Smith

# Assignment 2: Decision Trees

In this assignment we'll implement the Decision Tree algorithm to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains features from the Messidor image set to predict whether an image contains signs of diabetic retinopathy or not. This dataset has `1150` records and `20` attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

Attribute Information:

0) The binary result of quality assessment. 0 = bad quality 1 = sufficient quality.

1) The binary result of pre-screening, where 1 indicates severe retinal abnormality and 0 its lack. 

2-7) The results of MA detection. Each feature value stand for the number of MAs found at the confidence levels alpha = 0.5, . . . , 1, respectively. 

8-15) contain the same information as 2-7) for exudates. However, as exudates are represented by a set of points rather than the number of pixels constructing the lesions, these features are normalized by dividing the 
number of lesions with the diameter of the ROI to compensate different image sizes. 

16) The euclidean distance of the center of the macula and the center of the optic disc to provide important information regarding the patient's condition. This feature is also normalized with the diameter of the ROI.

17) The diameter of the optic disc. 

18) The binary result of the AM/FM-based classification.

19) Class label. 1 = contains signs of Diabetic Retinopathy, 0 = no signs of Diabetic Retinopathy.

#### Implementation: 
The function prototypes are given to you, please don't change those. You can add additional helper functions if needed. 

*Suggestion:* The dataset is substantially big, for the purpose of easy debugging, work with a subset of the data and test your decision tree implementation on that.

#### Notes:
Parts of this assignment will be **autograded** so a couple of caveats :-
- Entropy is calculated using log with base 2, `math.log2(x)`.
- For continuous features ensure that the threshold value lies exactly between 2 values. For example, if for feature 2 the best split occurs between 10 and 15 then the threshold value will be set as 12.5. For binary features [0/1] the threshold value will be 0.5.
- All values < `thresh_val` go to the left child and all values >= `thresh_val` go to the right child.

In [63]:
# Standard Headers
# You are welcome to add additional headers if you wish
# EXCEPT for scikit-learn... You may NOT use scikit-learn for this assignment!
import pandas as pd
import numpy as np
from math import log2

In [64]:
class DataPoint:
    def __str__(self):
        return "< " + str(self.label) + ": " + str(self.features) + " >"
    
    def __init__(self, label, features):
        self.label = label # the classification label of this data point
        self.features = features # a list of feature values for this data point

Q1. Read data from a CSV file. Put it into a list of `DataPoints`.

In [65]:
def get_data(filename):
    data = []
    df = pd.read_csv(filename, header=None)
    
    for idx, row in df.iterrows() : 
        dp = DataPoint(label=row[19], features=row[0:19]) # 19 is the class label, whether someone has diabetic retinopathy
        data.append(dp)
    return data

In [66]:
class TreeNode:
    is_leaf = True          # boolean variable to check if the node is a leaf
    feature_idx = None      # index that identifies the feature
    thresh_val = None       # threshold value that splits the node
    prediction = None       # prediction class (only valid for leaf nodes)
    left_child = None       # left TreeNode (all values < thresh_val)
    right_child = None      # right TreeNode (all values >= thresh_val)
    
    def printTree(self, level=0):    # for debugging purposes
        if self.is_leaf:
            print ('-'*level + 'Leaf Node:      predicts ' + str(self.prediction))
        else:
            print ('-'*level + 'Internal Node:  splits on feature ' 
                   + str(self.feature_idx) + ' with threshold ' + str(self.thresh_val))
            self.left_child.printTree(level+1)
            self.right_child.printTree(level+1)

Q2. Implement the function `make_prediction` that takes the decision tree root and a `DataPoint` instance and returns the prediction label.

In [67]:
def make_prediction(tree_root, data_point):
    if tree_root.is_leaf : 
        return tree_root.prediction
    if data_point.features[tree_root.feature_idx] >= tree_root.thresh_val : 
        prediction = make_prediction(tree_root.right_child, data_point)
    else : 
         prediction = make_prediction(tree_root.left_child, data_point) 
    return prediction

Q3. Implement the function `split_dataset` given an input data set, a `feature_idx` and the `threshold` for the feature. `left_split` will have all values < `threshold` and `right_split` will have all values >= `threshold`.

In [68]:
def split_dataset(data, feature_idx, threshold):
    left_split = []
    right_split = []
    for dp in data :
        if dp.features[feature_idx] < threshold :
            left_split.append(dp)
        else :
            right_split.append(dp)
    return (left_split, right_split)

Q4. Implement the function `calc_entropy` to return the entropy of the input dataset.

In [69]:
def calc_entropy(data):
    yes = 0
    no = 0
    for dp in data :
        if (dp.label == 1) :
            yes = yes + 1
        else : 
            no = no + 1
    sum = yes + no

    if yes == 0 or no == 0 :
        return 0
    else : 
        return -(yes/sum) * log2(yes/sum) - (no/sum) * log2(no/sum)



Q5. Implement the function `calc_best_threshold` which returns the best information gain and the corresponding threshold value for one feature at `feature_idx`.

In [70]:
# For the sake of meeting the 63% accuracy goal, this method gets about 0.75% better accuracy.
def calc_best_threshold(data, feature_idx):
    best_info_gain = 0.0
    best_thresh = None
    parent_entropy = calc_entropy(data)

    for i in range(0, len(data) - 1) : 
        cur_thresh = (data[i].features[feature_idx] + data[i + 1].features[feature_idx])/2
        cur_info_gain = calc_gain_at_thresh(cur_thresh, data, feature_idx, parent_entropy)

        if cur_info_gain > best_info_gain : 
            best_info_gain = cur_info_gain
            best_thresh = cur_thresh

    return (best_info_gain, best_thresh)
            
def calc_gain_at_thresh(threshold, data, feature_idx, parent_entropy) :
    left, right = split_dataset(data, feature_idx, threshold)
    e_left = calc_entropy(left)
    e_right = calc_entropy(right)
    cur_entropy = ((len(left)/len(data)) * e_left) + ((len(right)/len(data)) * e_right)
    cur_info_gain = parent_entropy - cur_entropy
    return cur_info_gain


# This code is a bit less accurate, but only calculates thresholds when there is a class switch.
# This makes it run about a minute and a half faster (on our small data subset-- the gains would be much more significant 
# with more real-world data sets).

# def calc_best_threshold(data, feature_idx):
#     best_info_gain = 0.0
#     best_thresh = None
#     parent_entropy = calc_entropy(data)

#     # sort data by feature_idx
#     data = sorted(data, key=lambda x: x.features[feature_idx])

#     # Check if all the values for an attribute are the same
#     unique = set(dp.features[feature_idx] for dp in data)
#     if (len(unique) == 1) :
#         return best_info_gain, best_thresh

#     for index in range(len(data)) :
#         # Check if right and left node feature values differ
#         if (index > 0 and data[index-1].label != data[index].label) : 
#             if (data[index-1].features[feature_idx] == data[index].features[feature_idx]):
#                 # check to the left
#                 left_bound = index
#                 while (left_bound-2 > 0 and data[left_bound-2].features[feature_idx] 
#                             == data[left_bound-1].features[feature_idx]) :
                    
#                     left_bound = left_bound-1
#                 left_thresh = (data[left_bound-2].features[feature_idx] + data[left_bound-1].features[feature_idx])/2
#                 info_gain_left = calc_gain_at_thresh(left_thresh, data, feature_idx, parent_entropy)
#                 if info_gain_left > best_info_gain: 
#                     best_info_gain = info_gain_left
#                     best_thresh = left_thresh

#                 # check to the right
#                 right_bound = index
#                 while (right_bound + 1 < len(data) and data[right_bound].features[feature_idx] 
#                             == data[right_bound+1].features[feature_idx]) :
                    
#                     right_bound = right_bound+1
#                 right_thresh = (data[right_bound-2].features[feature_idx] + data[right_bound-1].features[feature_idx])/2
#                 info_gain_right = calc_gain_at_thresh(right_thresh, data, feature_idx, parent_entropy)
#                 if info_gain_right > best_info_gain: 
#                     best_info_gain = info_gain_right
#                     best_thresh = right_thresh
                
#             else : 
#                 cur_thresh = (data[index-1].features[feature_idx] + data[index].features[feature_idx])/2
#                 cur_info_gain = calc_gain_at_thresh(cur_thresh, data, feature_idx, parent_entropy)

#                 if cur_info_gain > best_info_gain : 
#                     best_info_gain = cur_info_gain
#                     best_thresh = cur_thresh
#     return (best_info_gain, best_thresh)

Q6. Implement the function `identify_best_split` which returns the best feature to split on for an input dataset, and also returns the corresponding threshold value.

In [71]:
def identify_best_split(data):
    if len(data) < 2:
        print("none time!")
        return (None, None)
    best_feature = 0.0
    best_thresh = 0.0
    best_info_gain = 0.0
    cur_threshold = 0.0 
    cur_info_gain = 0.0
    parent_entropy = calc_entropy(data)
    for ft in range(0, 19) :
        if data[0].features[ft] > 1 : 
            # This is a continuous feature, so we need to worry about thresholds 
            cur_info_gain, cur_threshold = calc_best_threshold(data, ft)
        else : 
            # This is a categorical feature. The threshold will always be 0.5 for these features
            cur_threshold = 0.5
            left, right = split_dataset(data, ft, cur_threshold)
            entropy_left = calc_entropy(left)
            entropy_right = calc_entropy(right)
            total_entropy = (len(left) / len (data) * entropy_left) + ((len(right) / len(data)) * entropy_right)
            cur_info_gain = parent_entropy - total_entropy
        if (cur_info_gain > best_info_gain) :
            best_info_gain = cur_info_gain
            best_feature = ft
            best_thresh = cur_threshold
    return (best_feature, best_thresh)

Q7. Implement the function `create_leaf_node` which returns a `TreeNode` with `is_leaf=True` and `prediction` set to whichever classification occurs most in the dataset at this node. If there is a tie, choose classification label 1 (has disease). 

In [72]:
def create_leaf_node(data):
    node = TreeNode()
    node.is_leaf=True

    yes = 0
    no = 0
    for dp in data : 
        if dp.label : 
            yes = yes + 1
        else : 
            no = no + 1
    if (yes >= no) :
        node.prediction = 1
    else :
        node.prediction = 0

    return node

Q8. Implement the `create_decision_tree` function. `max_levels` denotes the maximum height of the tree (for example if `max_levels = 1` then the decision tree will only contain the leaf node at the root). [Hint: this is where the recursion happens.]

In [73]:
def create_decision_tree(data, max_levels):
    my_feature, my_thresh = identify_best_split(data)
    left, right = split_dataset(data, my_feature, my_thresh)

    root_node = TreeNode()
    root_node.feature_idx = my_feature
    root_node.thresh_val = my_thresh
    root_node.is_leaf = False

    if (max_levels == 1) :
        return create_leaf_node(data) 
    root_node.left_child = tree_helper(left, 1, max_levels, root_node)
    root_node.right_child = tree_helper(right, 1, max_levels, root_node)
    
    return root_node

def tree_helper(data_sub, cur_height, max_levels, cur_node) : 
    # Base case - all pure OR we've reached max height
    # todo: what if 'all of the attributes are the same'
    child = TreeNode()
    if (cur_height == max_levels or calc_entropy(data_sub) == 0) : 
        return create_leaf_node(data_sub)
    child.is_leaf = False

    my_feature, my_thresh = identify_best_split(data_sub)

    left, right = split_dataset(data_sub, my_feature, my_thresh)
    child.feature_idx = my_feature
    child.thresh_val = my_thresh

    child.left_child = tree_helper(left, cur_height + 1, max_levels, child)
    child.right_child = tree_helper(right, cur_height + 1, max_levels, child)
    return child

Q9. Given a test set, the function `calc_accuracy` returns the accuracy of the classifier. You'll use the `make_prediction` function for this.

In [74]:
def calc_accuracy(tree_root, data):
    correct = 0
    for dp in range(len(data)) : 
        res = make_prediction(tree_root, data[dp])
        if (res == data[dp].label) :
            correct = correct + 1
     
    return correct / len(data)

Q10. Keeping the `max_levels` parameter as 10, use 5-fold cross validation to measure the accuracy of the model. Print the accuracy of the model.

In [75]:
# edit the code here - this is just a sample to get you started
import time

data = get_data("messidor_features.txt")
data_subsets = []
sub_len = len(data) // 5
bounds = []
bounds.append((0, sub_len))

sum = 0.0

# partition data into train_set and test_set
train_set = []
testing_set = []

for i in range(1, 5) :
    bounds.append((bounds[i-1][0] + sub_len, bounds[i-1][1] + sub_len))

for bound in bounds :
    data_subsets.append(data[bound[0]:bound[1]])

start = time.time()

for testing_idx in range (0,5):
    for training_idx in range(0, len(data_subsets)):
        if training_idx!=testing_idx :
            train_set.extend(data_subsets[training_idx])
        else:
            testing_set = data_subsets[training_idx]         
    
    print ('Training set size:', len(train_set))
    print ('Test set size    :', len(testing_set))
    cur_accuracy = (calc_accuracy(create_decision_tree(train_set, 10), testing_set))
    print("Accuracy at fold", testing_idx, ":", cur_accuracy)
    sum = sum + cur_accuracy
    # reset training set
    train_set = []
end = time.time()
print ('Time taken:', end - start)

accuracy = (sum/5)*100.0
print("Total accuracy on the test set is (drumroll): ", accuracy)

Training set size: 920
Test set size    : 230


Accuracy at fold 0 : 0.6217391304347826
Training set size: 920
Test set size    : 230
Accuracy at fold 1 : 0.6304347826086957
Training set size: 920
Test set size    : 230
Accuracy at fold 2 : 0.6739130434782609
Training set size: 920
Test set size    : 230
Accuracy at fold 3 : 0.6304347826086957
Training set size: 920
Test set size    : 230
Accuracy at fold 4 : 0.6434782608695652
Time taken: 181.8340048789978
Total accuracy on the test set is (drumroll):  63.99999999999999
